# Crypto currency extraction

- Retrieve historical data of a crypto currency (Bitcoin)
    - https://min-api.cryptocompare.com/data/histominute?fsym=BTC&tsym=USD&limit=10
- Analyze it and store the wanted parameters
- Plot the stored data

## 1. Historical data retrievement

The API used to retrieve historical data is **CryptoCompare** https://min-api.cryptocompare.com/. We chose this API because it provides many benefits for free :
1. Retrieve historical data at minute interval up to 7 days. For older times, interval becomes hourly or daily. 
2. Many crypto currencies available TODO give a list
3. Easy to use, no need to authenticate
4. High rate limits : https://min-api.cryptocompare.com/stats/rate/limit

In [1]:
import urllib.request
import pandas as pd
import json
import numpy as np
import datetime
import time
import os

## API config
BASE_URL         = "https://min-api.cryptocompare.com/data/histominute" # API base url
CURRENCY         = "USD" # Available currency in the API
APP_NAME         = "bitcoint_sent_analysis" # API advises us to give an app name in requests

### CHANGE THIS PARAMETER IF YOU WANT DIFFERENT CRYPTO 
file = open("current_crypto.txt", "r")
if file:
    CRYPTO_SYMBOL = file.read() 
file.close()
if CRYPTO_SYMBOL is None:
    CRYPTO_SYMBOL    = 'BTC' # ZIL | BTC | NEXO # Crypto currency retrieved from API
print(CRYPTO_SYMBOL)
MAX_LIMIT        = 2000 # maximum data retrieved per API request

## personal config
FILE_PATH        = "data/crypto/"+CRYPTO_SYMBOL # Relative path to historical data
SEP_CHAR         = '~' # character seperating dates from and to in filename
ENVS             = ['CRYPTO', 'MOST_RECENT_FILE', 'MOST_RECENT_FILE_LINE_COUNT', 'MOST_RECENT_TS'] # Stored in var.csv
print(ENVS)
MAX_ROW_PER_FILE = 10000 # Each file storing data has a maximum amount of rows

BTC
['CRYPTO', 'MOST_RECENT_FILE', 'MOST_RECENT_FILE_LINE_COUNT', 'MOST_RECENT_TS']


In [2]:
def get_data_minutely(toTs, limit, crypto_symbol):
    '''
    Get data minutely
    
    --- params
    @ toTs = the data to return until this time. set to -1 if you want to retrieve most recent data
    @ limit = number of minutes to retrieve from 1 to 2000
    @ crypto_symbol = the symbol of cryptocurrency (e.g. 'BTC' for bitcoin)
    '''
    #if toTs < 0:
    #    contents = urllib.request.urlopen(
    #        f"{BASE_URL}?fsym={crypto_symbol}&tsym={CURRENCY}&limit={limit}&extraParams={APP_NAME}"
    #    ).read()
    #else:
    url_check=BASE_URL+"?fsym="+'BTC'+"&tsym="+CURRENCY+"&limit="+str(limit)+"&toTs="+str(-1)+"&extraParams="+APP_NAME
    print(url_check)
    contents = urllib.request.urlopen(url_check).read()
    json_string = contents.decode("utf-8")
    obj = json.loads(json_string)
    df = pd.DataFrame.from_dict(obj['Data'])
    if not df.empty:
        return df.drop(['volumefrom', 'volumeto'], axis=1)
    return df

def get_var(key, crypto):
    df_var = pd.read_csv("data/crypto/var_crypto.csv", sep=',',dtype={'MOST_RECENT_TS': np.int32, 'MOST_RECENT_FILE_LINE_COUNT': np.int32})
    return df_var[key].loc[df_var['CRYPTO'] == crypto].values[0]

def update_var(key, value, crypto):
    df_var = pd.read_csv("data/crypto/var_crypto.csv", sep=',',dtype={'MOST_RECENT_TS': np.int32, 'MOST_RECENT_FILE_LINE_COUNT': np.int32})
    df_var[key].loc[df_var['CRYPTO'] == crypto] = str(value)
    df_var.to_csv("data/crypto/var_crypto.csv", index=False)
    
def add_new_crypto(crypto):
    df_var = pd.read_csv("data/crypto/var_crypto.csv", sep=',', dtype={'MOST_RECENT_TS': np.int32, 'MOST_RECENT_FILE_LINE_COUNT': np.int32})
    if df_var[ENVS[0]].loc[df_var['CRYPTO'] == crypto].empty:
        new_line = pd.DataFrame([[crypto,-1,0,0]], columns=ENVS)
        df_var = df_var.append(new_line)
        df_var.to_csv("data/crypto/var_crypto.csv", index=False)

In [3]:
add_new_crypto(CRYPTO_SYMBOL)

### 1.1 Data understanding

The data returned contains the following information :

- time : unix timestamp when data was recorded
- open : The price at opening time of the interval
- close : The price at close time of the interval
- low : The lowest price observed during the interval
- high : The higher price reached during the interval
- volumefrom : TODO
- volumeto : TODO

In [4]:
df_histo = get_data_minutely(-1, 10, CRYPTO_SYMBOL)
print(df_histo['time'].iloc[-1])
print(datetime.datetime.fromtimestamp(int(df_histo['time'].iloc[0])).strftime('%Y-%m-%d %H:%M:%S'))
print(datetime.datetime.fromtimestamp(int(df_histo['time'].iloc[-1])).strftime('%Y-%m-%d %H:%M:%S'))
print(df_histo.shape)
df_histo.head(3)
df_histo.to_csv()
print(df_histo)

https://min-api.cryptocompare.com/data/histominute?fsym=BTC&tsym=USD&limit=10&toTs=-1&extraParams=bitcoint_sent_analysis
1618254480
2021-04-13 00:28:00
2021-04-13 00:38:00
(11, 7)
          time      high       low      open     close conversionType  \
0   1618253880  59949.49  59925.96  59925.96  59949.21         direct   
1   1618253940  59965.66  59949.14  59949.21  59958.32         direct   
2   1618254000  59958.51  59918.78  59958.32  59925.38         direct   
3   1618254060  59941.89  59918.59  59925.38  59927.75         direct   
4   1618254120  59927.75  59907.94  59927.75  59907.94         direct   
5   1618254180  59927.92  59896.23  59907.94  59927.92         direct   
6   1618254240  59941.29  59918.52  59927.92  59921.00         direct   
7   1618254300  59946.95  59921.00  59921.00  59946.95         direct   
8   1618254360  59967.79  59946.95  59946.95  59967.79         direct   
9   1618254420  59970.73  59960.92  59967.79  59960.98         direct   
10  1618254480  5

### 1.2 Collect and save data

Data is collected from the API and then saved in csv files. Each csv file contains a maximum amount of rows. We split the data into multiple files if this max amount is reached.

The API have limitations depending the time interval between two currencies. The more precise route for historical data let us retrieve currencies every minute but is limited to 7 days in the past. To retrieve more data back in the past, we must retrieve data hourly or daily.

In [5]:
#### Nummber of objects wished to be retrieved from API. 
# If using minutely, then it corresponds to the number of minutes
total_wished = 60 * 24 *7# API LIMIT is 7 days minutely which is 60 * 24 * 7 minutes

# Last unix timestamp to return data for
toTs = -1

# Init dataframe of historical data
df_historical = pd.DataFrame()

# Retrieve existing most recent file if any
# If any file is present and it is not full, we'll use it to store future data
last_ts = get_var(ENVS[3], CRYPTO_SYMBOL)
last_file_line_cnt = 0
last_file = None
if last_ts > 0:
    last_file = get_var(ENVS[1], CRYPTO_SYMBOL)
    last_file_line_cnt = get_var(ENVS[2], CRYPTO_SYMBOL)
    now = int(time.time())
    total_wished = int((now - last_ts)/60) # Get data we didn't retrieve from now until last time

# Retrieve data for the 7 past days until last_ts
while(total_wished > 0):
    limit = min(total_wished, MAX_LIMIT)
    df = get_data_minutely(toTs, limit, CRYPTO_SYMBOL)
    #if df.empty:
    #    print(df.shape)
    #    break
    df_historical = df_historical.append(df)
    min_time = df['time'].iloc[0]
    #min_time=1616859340
    print(min_time)
    max_time = df['time'].iloc[-1]
    print(max_time)
    print('range=', datetime.datetime.fromtimestamp(min_time).strftime('%Y-%m-%d %H:%M:%S'),
         datetime.datetime.fromtimestamp(max_time).strftime('%Y-%m-%d %H:%M:%S'))
    toTs = min_time
    total_wished = total_wished - df.shape[0]
# Sort data in ascending time
if not df_historical.empty:

    df_historical = df_historical.sort_values(by=['time'])

    print(last_file_line_cnt)
    ## Save historical data to csv
    available_lines = MAX_ROW_PER_FILE - last_file_line_cnt
    df_to_fulfill = df_historical.head(available_lines)
    df_remaining  = df_historical.tail(df_historical.shape[0] - df_to_fulfill.shape[0])

    print('df_to_fulfill', df_to_fulfill.shape)
    print('df_remaining', df_remaining.shape)

    # Fulfill last file if any
    if not df_to_fulfill.empty:
        from_str = datetime.datetime.fromtimestamp(df_to_fulfill['time'].iloc[0]).strftime('%Y-%m-%d %H-%M-%S')
        max_time = df_to_fulfill['time'].iloc[-1]
        to_str = datetime.datetime.fromtimestamp(max_time).strftime('%Y-%m-%d %H-%M-%S')
        if last_file is None:
            last_file = FILE_PATH+"/"+from_str+SEP_CHAR+to_str+".csv"
            df_to_fulfill.to_csv(last_file, mode='w', header=True, index=False)
        else:
            print(last_file)
            df_to_fulfill.to_csv(last_file, mode='a', header=False, index=False)
            #filename = last_file.split(SEP_CHAR)[0]+SEP_CHAR+to_str+".csv"
            #print(filename)
            filename = last_file.split('~')[0]+SEP_CHAR+to_str+".csv"
            os.rename(last_file, filename)
        last_file_line_cnt = last_file_line_cnt + df_to_fulfill.shape[0]
        
    # Split remaining dataframe into chunk of MAX_ROW_PER_FILE rows max
    list_df_remaining = [df_remaining[i:i+MAX_ROW_PER_FILE] for i in range(0, df_remaining.shape[0], MAX_ROW_PER_FILE)]
    for df_remain in list_df_remaining:
        from_str = datetime.datetime.fromtimestamp(df_remain['time'].iloc[0]).strftime('%Y-%m-%d %H-%M-%S')
        max_time = df_remain['time'].iloc[-1]
        to_str = datetime.datetime.fromtimestamp(max_time).strftime('%Y-%m-%d %H-%M-%S')
        filename = FILE_PATH+"/"+from_str+SEP_CHAR+to_str+".csv"
        #filename = "data/crypto/BTC/"+from_str+SEP_CHAR+to_str+".csv"
        print(filename)
        df_remain.to_csv(filename, index = False)
        last_file_line_cnt = df_remain.shape[0]

    # Update env variables
    update_var(ENVS[3], max_time, CRYPTO_SYMBOL)
    update_var(ENVS[1], filename, CRYPTO_SYMBOL)
    update_var(ENVS[2], last_file_line_cnt, CRYPTO_SYMBOL)

print('done')

https://min-api.cryptocompare.com/data/histominute?fsym=BTC&tsym=USD&limit=1494&toTs=-1&extraParams=bitcoint_sent_analysis
1618164840
1618254480
range= 2021-04-11 23:44:00 2021-04-13 00:38:00
3211
df_to_fulfill (1495, 7)
df_remaining (0, 7)
data/crypto/BTC/2021-04-09 18-16-00~2021-04-11 23-44-00.csv
done


/home/mrinalini/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## 2. Plot crypto currency time series data

Read and plot retrieved data

In [6]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True) # plotly offline mode

import glob

MAX_ROW = 500 # Plot the MAX_ROW most recent points

### 2.1 Read data from csv files

For this example we'll collect max 1000 points (the most recent ones)

In [7]:
def get_most_recent_data_from_csv(folder, n_rows):
    '''
    @ folder : relative path where the data is contained
    @ n_rows : the number of rows to retrieve up to the most recent ones
    
    Return a dataframe containing n_rows of the most recent data retrieved
    '''
    files =  glob.glob("data/crypto/BTC/*.csv")
    print(files)
    files = sorted(files)
    df = pd.DataFrame()
    for file in reversed(files):
        print(file)
        df = df.append(pd.read_csv(file))
        if df.shape[0] > n_rows:
            break
    return df.sort_values(by=['time']).tail(n_rows)

In [8]:
# Read MAX_POINTS from the last file(s)
df_hist_minutes = get_most_recent_data_from_csv(FILE_PATH, MAX_ROW)
print('df shape', df_hist_minutes.shape)
from_str = datetime.datetime.fromtimestamp(df_hist_minutes['time'].iloc[0]).strftime('%Y-%m-%d %H-%M-%S')
to_str = datetime.datetime.fromtimestamp(df_hist_minutes['time'].iloc[-1]).strftime('%Y-%m-%d %H-%M-%S')
print('from', from_str, 'to', to_str)
df = df_hist_minutes
df['time'] = df_hist_minutes['time'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
df.head(5)

['data/crypto/BTC/2021-04-09 18-16-00~2021-04-13 00-38-00.csv', 'data/crypto/BTC/2021-03-25 13-23-00~2021-03-26 22-30-00.csv', 'data/crypto/BTC/2021-04-02 19-43-00~2021-04-09 18-15-00.csv', 'data/crypto/BTC/2021-03-26 22-30-00~2021-04-02 19-42-00.csv']
data/crypto/BTC/2021-04-09 18-16-00~2021-04-13 00-38-00.csv
df shape (500, 7)
from 2021-04-12 16-19-00 to 2021-04-13 00-38-00


,time,high,low,open,close,conversionType,conversionSymbol
4206,2021-04-12 16:19:00,59753.24,59726.96,59750.27,59744.49,direct,NaN
4207,2021-04-12 16:20:00,59749.09,59657.29,59744.49,59657.29,direct,NaN
4208,2021-04-12 16:21:00,59692.98,59636.82,59657.29,59673.91,direct,NaN
4209,2021-04-12 16:22:00,59675.90,59639.11,59673.91,59664.25,direct,NaN
4210,2021-04-12 16:23:00,59716.54,59659.31,59664.25,59716.54,direct,NaN


### 2.2 Plot candlestick like graph

Plot only the most recent points to see if evereything did well.

In [9]:
trace = go.Candlestick(x=df.time,
                       open=df.open,
                       high=df.high,
                       low=df.low,
                       close=df.close)
data = [trace]
layout = {
    'title': f"{CRYPTO_SYMBOL} currency",
    'yaxis': {'title': 'Price USD'},
}
fig = dict(data=data, layout=layout)
iplot(fig)